# Применение инструментов Hugging face и предобученных моделей

### Вариант 1:
 Вам нужно создать искусственные данные для тестирования и/или обучения чат-бота. По заданному предложению/утверждению/команде создать набор расширенных предложений/утверждений/команд с приблизительно тем же смыслом. Пример:

> After your workout, remember to focus on maintaining a good water balance.

похожие команды:

> Remember to drink enough water to restore and maintain your body's hydration after your cardio training.

>Please don't forget to maintain water balance after workout.

Предлагается решить упрощенную версию данной задачи с применением общедоступных "маленьких". 
В репозитории Hugging Face есть большое количество предобученных моделей для [casual](https://huggingface.co/models?pipeline_tag=text-generation) и [masked](https://huggingface.co/models?pipeline_tag=fill-mask) языкового моделирования.  Также для валидации можно использовать [sentence-transformers](https://huggingface.co/sentence-transformers). Выбрать нужно модели, которые можно запускать на CPU.

Пример использования masked LM:

```python
import torch
from transformers import AutoModelForMaskedLM, PreTrainedTokenizer

# загружается токенайзер
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# загружается модель
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

# предложение и замаскированным токеном
sequence = f"My name is {tokenizer.mask_token}."

# результат токенизации
input_ids = tokenizer.encode(sequence, return_tensors="pt")
# применение модели
result = model(input_ids=input_ids)

# индекс замаскированного токена (NB может не совпадать с номером слова)
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

# самый вероятный токен 
print(tokenizer.decode(result.logits[:, mask_token_index].argmax()))
```

или через [pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)

```python
from transformers import pipeline

pipe = pipeline("fill-mask", model="distilroberta-base")

pipe("My name is <mask>.")
```

Casual LM через pipeline:

```python
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')

generator("Hello", max_length=10, num_return_sequences=5)
```

Один наивных способов решения задачи без дополнительного обучения - замаскировать, или вставить в исходную команду замаскированный токен, или обрезать часть команды и применить языковую модель. Результат можно валидировать с помощью [sentence-transformers](https://huggingface.co/sentence-transformers). 

### Вариант 2:

Нужно реализовать простейшую семантическую поисковую систему помощью векторного представления предложений/текстов.
1. Выбрать коллекцию текстовых документов (небольшое подмножество статей из Википедии (из дампа), новости, и т.п.).
2. Выбрать модель для получения векторных представлений (например [sentence-transformers](https://huggingface.co/sentence-transformers)).
3. Выбрать векторное хранилище (faiss, lancedb, qdrant, chroma, pgvector, redis и т.д.)
4. Реализовать поиск, (возможно с постфильтрацией) и продемонстрировать его работу. Индексация и поиск должны быть реализованы в виде отдельных скриптов с CLI.

Нельзя использовать LangChain. 


# Вариант 1

Загрузка и подготовка данных

In [3]:
from datasets import load_dataset

# Загружаем датасет
dataset = load_dataset("snips_built_in_intents")

# Мы будем работать с тренировочным набором данных
train_data = dataset["train"]


Generating train split: 100%|██████████| 328/328 [00:00<00:00, 10760.09 examples/s]


Генерация данных с помощью GPT-2

In [12]:
from transformers import pipeline, set_seed

# Инициализация модели GPT-2
generator = pipeline('text-generation', model='gpt2')

# Функция для генерации искусственных данных
def generate_data(text, num_sequences=5):
    return generator(text, max_length=25, num_return_sequences=num_sequences)

# Пример генерации данных для одного предложения
example = train_data[0]['text']  # Пример из датасета
generated_examples = generate_data(example)

print(generated_examples)

print(len(train_data))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Share my location with Hillary\'s sister\'s help. I need our help."\n\nMs. Johnson, a member of the'}, {'generated_text': "Share my location with Hillary's sister and my sister's favorite food! (Photo by Justin Sullivan/Getty Images)\n\n"}, {'generated_text': "Share my location with Hillary's sister, who went to an airport where the FBI was located\n\n(APPLAUSE)"}, {'generated_text': 'Share my location with Hillary\'s sister," Clinton said before the rally, saying that "Clinton is up there with her sister,'}, {'generated_text': "Share my location with Hillary's sister\n\nHillary's sister did not have to get out of the basement where they were supposed"}]
328


In [9]:
from transformers import pipeline
from datasets import load_dataset
import pandas as pd

# Загружаем датасет
dataset = load_dataset("snips_built_in_intents")

# Инициализация модели GPT-2
generator = pipeline('text-generation', model='gpt2')

# Функция для генерации искусственных данных
def generate_data(text, num_sequences=5):
    return generator(text, max_length=50, num_return_sequences=num_sequences)

# Генерация искусственных данных для всего датасета
generated_dataset = []
for example in dataset["train"]:
    original_text = example['text']
    generated_examples = generate_data(original_text)
    for g in generated_examples:
        generated_dataset.append({'original': original_text, 'generated': g['generated_text']})

# После генерации, generated_dataset содержит исходные и сгенерированные тексты
# Преобразование в DataFrame
df = pd.DataFrame(generated_dataset)

# Сохранение в CSV файл
df.to_csv('generated_dataset.csv', index=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [17]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset

# Загружаем датасет
dataset = load_dataset("snips_built_in_intents")

# Инициализация модели GPT-2
generator = pipeline('text-generation', model='gpt2')

# Функция для обрезания текста и генерации искусственных данных
def generate_data(text, num_sequences=3):
    # Обрезаем текст до первой половины
    cut_point = len(text) // 2
    shortened_text = text[:cut_point]
    return generator(shortened_text, max_length=30, num_return_sequences=num_sequences)

# Генерация искусственных данных для всего датасета
generated_dataset = []
for example in dataset["train"]:
    original_text = example['text']
    generated_examples = generate_data(original_text)
    for g in generated_examples:
        generated_dataset.append({'original': original_text, 'generated': g['generated_text']})

# Преобразование в DataFrame
df = pd.DataFrame(generated_dataset)

# Сохранение в CSV файл
df.to_csv('generated_dataset2.csv', index=False)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
# import pandas as pd
# from transformers import pipeline
# from datasets import load_dataset

# # Загружаем датасет
# dataset = load_dataset("snips_built_in_intents")

# # Инициализация модели GPT-2
# generator = pipeline('text-generation', model='gpt2')

# # Функция для обрезания текста и генерации искусственных данных
# def generate_data(original_text, num_sequences=3):
#     # Обрезаем текст до первой половины
#     cut_point = len(original_text) // 2
#     shortened_text = original_text[:cut_point]
#     generated_examples = generator(shortened_text, max_length=20, num_return_sequences=num_sequences)
#     return generated_examples

# # Генерация искусственных данных для всего датасета
# generated_dataset1 = []
# for example in dataset["train"]:
#     original_text = example['text']
#     generated_examples = generate_data(original_text)
#     for g in generated_examples:
#         generated_dataset1.append({'original': original_text, 'generated': g['generated_text']})

# # Преобразование в DataFrame
# df1 = pd.DataFrame(generated_dataset1)

# # Сохранение в CSV файл
# df1.to_csv('generated_dataset1.csv', index=False)


Валидация сгенерированных данных

In [20]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Загрузка модели для семантического сравнения
model = SentenceTransformer('all-mpnet-base-v2')

# Загрузка датасета (предполагается, что вы уже сгенерировали и сохранили его)
df = pd.read_csv('generated_dataset.csv')

# Функция для вычисления косинусного сходства между исходным и сгенерированным текстами
def validate_data(original, generated):
    original_emb = model.encode(original, convert_to_tensor=True)
    generated_emb = model.encode(generated, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(original_emb, generated_emb)
    return cosine_scores.item()

# Применяем функцию валидации к каждой паре исходного и сгенерированного текста
df['similarity'] = df.apply(lambda row: validate_data(row['original'], row['generated']), axis=1)


In [21]:
print(df['similarity'])

0       0.641652
1       0.622939
2       0.710083
3       0.858834
4       0.670564
          ...   
1635    0.752843
1636    0.689018
1637    0.802743
1638    0.413657
1639    0.661435
Name: similarity, Length: 1640, dtype: float64


In [22]:
scores_df = pd.DataFrame(df)

# Сохранение в CSV файл
scores_df.to_csv('scores2.csv', index=False)

In [26]:
# df1 = pd.read_csv('generated_dataset1.csv')
# # Применяем функцию валидации к каждой паре исходного и сгенерированного текста
# df1['similarity'] = df1.apply(lambda row: validate_data(row['original'], row['generated']), axis=1)

# scores_df1 = pd.DataFrame(df1)
# scores_df1.to_csv('scores1.csv', index=False)

# Вариант 1 (masked)

In [29]:
import random
import re
from transformers import AutoTokenizer

# Инициализация токенайзера
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

def my_masking_method(text, mask_token):
    # Разделение текста на слова
    words = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)

    # Выбор случайного набора слов для маскирования
    maskable_words = [word for word in words if word.isalnum()]  # Исключение знаков препинания и специальных символов
    num_to_mask = max(1, len(maskable_words) // 4)  # 25% слов, но хотя бы одно

    words_to_mask = random.sample(maskable_words, num_to_mask)

    # Замена выбранных слов на маски
    masked_text = text
    for word in words_to_mask:
        masked_text = masked_text.replace(word, mask_token, 1)  # Заменяем каждое слово на маску по одному разу

    return masked_text

# Пример использования
example_text = "After your workout, remember to focus on maintaining a good water balance."
masked_example = my_masking_method(example_text, tokenizer.mask_token)
print(masked_example)


config.json: 100%|██████████| 480/480 [00:00<00:00, 69.0kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 1.05MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.14MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:01<00:00, 826kB/s]


After your <mask>, remember <mask> focus on maintaining a good water <mask>.


In [36]:
import torch
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import random
import re

# Инициализация токенайзера и модели для MLM
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Функция для маскирования текста
def my_masking_method(text, mask_token):
    words = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
    maskable_words = [word for word in words if word.isalnum()]
    num_to_mask = max(1, len(maskable_words) // 4)  # 25% слов
    words_to_mask = random.sample(maskable_words, num_to_mask)
    masked_text = text
    for word in words_to_mask:
        masked_text = masked_text.replace(word, mask_token, 1)
    return masked_text

# Загрузка датасета
dataset = load_dataset("snips_built_in_intents")

# Генерация маскированных данных
generated_dataset = []
for example in dataset["train"]:
    original_text = example['text']
    masked_text = my_masking_method(original_text, tokenizer.mask_token)
    generated_examples = fill_mask(masked_text, top_k=7)
    for g in generated_examples:
        if 'sequence' in g:  # Проверяем, есть ли ключ 'sequence' в словаре
            generated_dataset.append({'original': original_text, 'generated': g['sequence']})


# Загрузка модели для семантического сравнения
sbert_model = SentenceTransformer('all-mpnet-base-v2')

# Функция валидации данных
def validate_data(original, generated):
    original_emb = sbert_model.encode(original, convert_to_tensor=True)
    generated_emb = sbert_model.encode(generated, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(original_emb, generated_emb)
    return cosine_scores.item()

# Валидация сгенерированных данных
for item in generated_dataset:
    item['similarity'] = validate_data(item['original'], item['generated'])

# Преобразование в DataFrame и сохранение в файл
import pandas as pd
df = pd.DataFrame(generated_dataset)
df.to_csv('validated_generated_dataset.csv', index=False)


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
print(len(df))

623
